In [10]:
import pandas as pd
import numpy as np

## Shakira
Shakira M: 15951.0

In [58]:
with open('../data/raw/lightdump/es_wiki.txt') as fh:
    shakira = []
    found = False
    for line in fh:
        if found and (line[0] != '^'):
            break
        if found:
            shakira.append(line.strip())
        if line.strip() == 'Shakira':
            found = True

In [59]:
shakira = shakira[::-1]

In [68]:
shakira[len(shakira)-1]

'^^^_2010-10-20T17:15:23Z 0 3816 Seebiitaaw'

## Chile
Chile M: 332595.0

In [37]:
with open('../data/raw/lightdump/es_wiki.txt') as fh:
    chile = []
    found = False
    for line in fh:
        if found and (line[0] != '^'):
            break
        if found:
            chile.append(line.strip())
        if line.strip() == 'Chile':
            found = True

In [38]:
chile = chile[::-1]

## M stat

In [66]:
revert = 0
revert_pairs = []

#the list of mutual reverting pairs
mutual_revert_pairs = []
#the list of unique users among reverting pairs
mutual_revert_users = []

#a dictionary of user as key, and his number of edits as the value
user_edits = {}

#a dictionary with the line number (actual version number) as key, and the line label as value(i.e. line label is either the same as version number if not revert version, or equal to an older version number if it's a revert version)
lineLabels = []
#a dictionary with the line number (actual version number) as key, and the author of that line as value
lineAuthors = []

def getMutual():
    global revert_users
    global mutual_revert_users

    for pair in revert_pairs:
        parts = pair.split("~!~")
        if parts[1] + "~!~" + parts[0] in revert_pairs:
            sorted_pair = ""
            if parts[0] < parts[1]:
                sorted_pair = parts[0] + "~!~" + parts[1]
            else:
                sorted_pair = parts[1] + "~!~" + parts[0]
                mutual_revert_pairs.append(sorted_pair)
            if parts[1] not in mutual_revert_users:
                mutual_revert_users.append(parts[1])
            if parts[0] not in mutual_revert_users:
                mutual_revert_users.append(parts[0])


def getLine(label):
    global lineLabels
    for line, ll in reversed(list(enumerate(lineLabels))):
        if lineLabels[line] == label:
            return line

def readFile():
    global user_edits
    global revert_pairs
    global lineLabels
    global lineAuthors
    global revert
    for ln in shakira:
        parts = ln.strip().split(' ');

        if parts[3] not in user_edits:
            user_edits[parts[3]] = 1
        else:
            user_edits[parts[3]] = user_edits[parts[3]] + 1
        if parts[1] == '1':
            revert += 1
            #the found line is the version i-1 equal to this version j, and the revert is assumed to be between the author of i, and j
            line = getLine(int(parts[2]))
            #ignore cases when i-1, and i are equal (consecutive versions)
            if line >= len(lineLabels)-1:
                continue
            revertedU = lineAuthors[line + 1]
            revertingU = parts[3]
            if revertedU == revertingU:
                continue
            pair = revertedU + "~!~" + revertingU
            if pair not in revert_pairs:
                revert_pairs.append(pair)
        lineLabels.append(int(parts[2]))
        lineAuthors.append(parts[3])

#main part
readFile()

#calculating the score
getMutual()
score = 0
for pair in list(set(mutual_revert_pairs)):
    parts = pair.split("~!~")
    u1 = parts[0]
    u2 = parts[1]
    if user_edits[u1]<user_edits[u2]:
        edit_min = user_edits[u1]
    else:
        edit_min = user_edits[u2]
    print(pair + ":" + str(edit_min))
    score += edit_min

score *= len(mutual_revert_users)        
print(score)

Chien~!~Hernia_de_Disco:13
200.12.190.69~!~Matdrodes:2
Albes23~!~Camima:17
Edub~!~Kristen2007:3
83.52.172.94~!~Alhen:3
190.143.39.47~!~Chico512:37
83.60.48.222~!~Matdrodes:1
Chico512~!~Famous-top-secret:29
66.50.236.206~!~Isha:2
201.221.201.56~!~Manwë:5
189.133.94.15~!~Cobalttempest:13
4.234.15.92~!~FedericoMP:1
201.221.201.56~!~AVBOT:5
200.58.204.250~!~Varano:1
81.37.255.93~!~Matdrodes:1
83.42.114.32~!~Matdrodes:1
Albes23~!~Chico512:17
83.36.122.64~!~Pinar:1
Camima~!~Seebiitaaw:9
81.39.127.56~!~Matdrodes:1
190.51.161.162~!~Matdrodes:2
83.32.245.130~!~Matdrodes:1
83.52.172.182~!~Zero_Gravity:2
5845


# EDA
Shakira

In [69]:
s_df = pd.Series(shakira).str.split(' ', expand=True)
s_df.columns = ['timestamp', 'revert', 'revID', 'username']

In [70]:
s_df[s_df.username.str.contains('189.133.94.15') | s_df.username.str.contains('Cobalttempest')].head()

,timestamp,revert,revID,username
2420,^^^_2007-11-10T02:49:51Z,1,2020,Cobalttempest
2422,^^^_2007-11-10T03:09:25Z,1,2020,Cobalttempest
2424,^^^_2007-11-10T03:19:04Z,1,2020,Cobalttempest
2435,^^^_2007-11-11T16:59:43Z,1,2030,Cobalttempest
2441,^^^_2007-11-11T22:18:26Z,1,2030,Cobalttempest


In [74]:
obs = s_df.iloc[2608:2658].username.value_counts(normalize=True).iloc[:2].sum()
obs

0.32

In [72]:
np.random.seed(1)
rand_nums = np.random.choice(range(0, len(shakira) - 200), size=500)
props = []
for i in rand_nums:
    props.append(s_df.iloc[i:i+50].username.value_counts(normalize=True).iloc[:2].sum())

In [73]:
np.mean(props)

0.26676000000000005

In [75]:
(np.array(props) > 0.32).mean()

0.208

Chile

In [41]:
c_df = pd.Series(chile).str.split(' ', expand=True)
c_df.columns = ['timestamp', 'revert', 'revID', 'username']

In [53]:
c_df[c_df.username.str.contains('Investigadore') | c_df.username.str.contains('B1mbo')].iloc[400:600]

,timestamp,revert,revID,username
4342,^^^_2007-12-02T20:27:03Z,0,3414,B1mbo
4343,^^^_2007-12-02T20:31:37Z,0,3415,B1mbo
4345,^^^_2007-12-10T20:06:05Z,0,3417,B1mbo
4353,^^^_2007-12-17T19:20:08Z,1,3422,B1mbo
4354,^^^_2007-12-17T19:21:45Z,0,3424,B1mbo
4359,^^^_2007-12-18T22:22:54Z,0,3428,B1mbo
4360,^^^_2007-12-18T22:32:51Z,0,3429,B1mbo
4364,^^^_2007-12-25T19:00:46Z,1,3431,B1mbo
4374,^^^_2008-01-10T00:21:57Z,0,3440,B1mbo
4375,^^^_2008-01-10T01:14:50Z,0,3441,B1mbo


In [79]:
obs = c_df.iloc[4750:4800].username.value_counts(normalize=True).iloc[:2].sum()
obs

0.64

In [80]:
np.random.seed(1)
rand_nums = np.random.choice(range(0, len(chile) - 50), size=500)
props = []
for i in rand_nums:
    props.append(c_df.iloc[i:i+50].username.value_counts(normalize=True).iloc[:2].sum())

In [81]:
(np.array(props) > obs).mean()

0.086

In [82]:
np.mean(props)

0.37244